## Data Reading and Transformation
The data (BCI Arboreal) has a lot of fields which are NaN and a lot of irrelevant variables for my analysis right now I only require location (X, Y, UTM-north, UTM-east) 
Time(timestamp)
Name fields

So I run queries to filter out the above fields and remove any empty data fields from the dataframe

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/Dead-Reckoning Arboreal Species in BCI.csv')
# convert the timestamp column to Unix timestamps
df['timestamp'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.timestamp())

# convert the Unix timestamps to integers
df['timestamp'] = df['timestamp'].astype(int)

/var/folders/2z/k0vctg855l3dq626nd2hf5j40000gn/T/ipykernel_29595/2044101010.py:3: DtypeWarning: Columns (7,9,18,32,37,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Dead-Reckoning Arboreal Species in BCI.csv')


After reading the data. I'm only taking the GPS sensor logs. I tried to merge the records of other sensors (acc / mag). The only reasonable field I thought would be timestamp however the records didn't match good enough so only keeping the GPS data for now

In [2]:
##Considering the GPS data for this study
df_gps = df[df['sensor-type'] == 'gps']
# df_gps.dropna(axis = 1, inplace = True)|
df_extract = df_gps[['utm-easting', 'utm-northing', 'timestamp']]

6 steps for generalizing and summarizing data to make a flow map 

4. use the resulting Voronoi cells as places for place-based division of the trajectories into segments;
5. for each ordered pair of places, aggregate the trajectory segments starting in the first place and ending in the second place;
6. measure the quality of the generalization and improve it if necessary.

### 1. extract characteristic points from the trajectories;


In [3]:
import math

def get_mean(lst):
    return sum(lst) / len(lst)

def spatial_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def angle_between_vectors(v1, v2):
    dot_product = v1[0] * v2[0] + v1[1] * v2[1]
    magnitude_product = math.sqrt(v1[0]**2 + v1[1]**2) * math.sqrt(v2[0]**2 + v2[1]**2)
    return math.degrees(math.acos(dot_product / magnitude_product))

def extract_significant_points(points, min_distance = 3, max_distance = 10, min_angle = 30, min_stop_duration = 100):
    n = len(points)
    C = [points[0]]
    i = 0
    j = 1

    while j < n:
        
        if j == n - 1:
            C.append(points[j])
            break

        d_space_i_j = spatial_distance(points[i][:2], points[j][:2])
        if d_space_i_j >= max_distance:
            C.append(points[j])
            i = j
            j = i + 1
        #Fine
        else:
            k = j + 1
            while k < n and spatial_distance(points[j][:2], points[k][:2]) < min_distance:
                k = k + 1
            #Fine
            if k > j + 1:
                d_time = points[k-1][2] - points[j][2]
                if d_time >= min_stop_duration:
                    C.append(points[j])
                    i = j
                    j = k
                    
            #Fine
                else:
                    x_vals, y_vals, t_vals = zip(*points[j:k])
                    x_ave = get_mean(x_vals)
                    y_ave = get_mean(y_vals)

                    distances = [spatial_distance((x, y), (x_ave, y_ave)) for x, y, _ in points[j:k]]
                    m = j + distances.index(min(distances))

                    C.append(points[m])
                    i = j
                    j = m + 1

            else:
                a_turn = angle_between_vectors((points[i][0] - points[j][0], points[i][1] - points[j][1]),
                                               (points[j][0] - points[k][0], points[j][1] - points[k][1]))
                if a_turn >= min_angle:
                    C.append(points[j])
                    i = j
                    j = k
                else:
                    j += 1

    return C
extracted_points = extract_significant_points(df_extract.to_numpy())


/var/folders/2z/k0vctg855l3dq626nd2hf5j40000gn/T/ipykernel_29595/2157936881.py:12: RuntimeWarning: invalid value encountered in double_scalars
  return math.degrees(math.acos(dot_product / magnitude_product))


In [4]:
len(extracted_points)

203996

### 2. Group the extracted points by spatial proximity;
### Class definitions

In [6]:
import math
from typing import List, Tuple, Set

class Point:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

class Group:
    def __init__(self, centroid: Point, members: Set[Point]):
        self.centroid = centroid
        self.members = members

class Grid:
    def __init__(self, x_min: float, x_max: float, y_min: float, y_max: float, max_radius: float):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.max_radius = max_radius
        self.n_rows = math.ceil((y_max - y_min) / max_radius)
        self.n_cols = math.ceil((x_max - x_min) / max_radius)
        self.cells = [[set() for _ in range(self.n_cols)] for _ in range(self.n_rows)]

    def get_grid_position(self, point: Point) -> Tuple[int, int]:
        i = math.floor((point.y - self.y_min) / self.max_radius)
        j = math.floor((point.x - self.x_min) / self.max_radius)
        return i, j

    def get_closest_centroid(self, point: Point) -> Group:
        i, j = self.get_grid_position(point)
        c = None
        for row in range(max(i - 1, 0), min(i + 2, self.n_rows)):
            for col in range(max(j - 1, 0), min(j + 2, self.n_cols)):
                for centroid in self.cells[row][col]:
                    if self._spatial_distance(point, centroid) <= self.max_radius:
                        if c is None or self._spatial_distance(point, centroid) < self._spatial_distance(point, c.centroid):
                            c = Group(centroid, set())
        return c

    def add_centroid(self, group: Group):
        i, j = self.get_grid_position(group.centroid)
        self.cells[i][j].add(group.centroid)

    def remove_centroid(self, group: Group):
        i, j = self.get_grid_position(group.centroid)
        self.cells[i][j].remove(group.centroid)

    def _spatial_distance(self, p1: Point, p2: Point) -> float:
        return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

    
# def redistribute_points(P, group, grid):
    
    
    
#     for group in groups:
#         group.members = [group.centroid]
        
        
    
#     for p in points:
#            c = G.get_closest_centroid(p)
#         if c is None:
#             g = Group(p, p)
#             R.add(g)
#     else:
#         g = next((g for g in R if g.centroid == c), None)
#         if g is not None:
#             print(g)
#             g.members.append(p)
#             G.remove_centroid(c)
#             g.centroid = get_centroid(g.members)
#     if g is not None:
#         i, j = G.get_grid_position(g.centroid)
#         G.add_centroid(g)

def put_in_proper_group(p: Tuple[float, float], R: List[Group], G: Grid) -> None:
    c = G.get_closest_centroid(p)
    if c is None:
        g = Group(p, p)
        R.add(g)
    else:
        g = next((g for g in R if g.centroid == c), None)
        if g is not None:
            print(g)
            g.members.append(p)
            G.remove_centroid(c)
            g.centroid = get_centroid(g.members)
    if g is not None:
        i, j = G.get_grid_position(g.centroid)
        G.add_centroid(g)
        

def build_groups(points: List[Point], max_radius: float) -> Set[Group]:
    # Step 1
    x_min = min(point.x for point in points)
    x_max = max(point.x for point in points)
    y_min = min(point.y for point in points)
    y_max = max(point.y for point in points)

    # Step 2
    grid = Grid(x_min, x_max, y_min, y_max, max_radius)

    # Step 3
    groups = set()

    i = 0
    # Step 4
    for point in points:
        if(i > 10000):
            print(i)
            i = 0
        i += 1
        put_in_proper_group(point, groups, grid)
    print(len(groups))
    # Step 5
#     redistribute_points(points, groups, grid)

    # Step 6
    return groups
extracted_points = [Point(point[0], point[1]) for point in extracted_points]
g1 = build_groups(extracted_points, 300)

10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
10001
42


In [ ]:
points = extracted_points
max_radius = 300
x_min = min(point.x for point in points)
x_max = max(point.x for point in points)
y_min = min(point.y for point in points)
y_max = max(point.y for point in points)

# Step 2
grid = Grid(x_min, x_max, y_min, y_max, max_radius)

# Step 3
groups = set()

i = 0
# Step 4
for point in points:
    if(i > 10000):
        print(i)
        i = 0
    i += 1
    put_in_proper_group(point, groups, grid)
print(len(groups))
# Step 5
# redistribute_points(points, groups, grid)

In [ ]:
grid.cells

### Converting extracted points to Point Class 

### Functions - already working code (but time C too high)

## 2.1 Visualizing Groups

In [ ]:
groups = g1
import matplotlib.pyplot as plt
from matplotlib import cm
colors = cm.rainbow([i/len(groups) for i in range(len(groups))])
for i, group in enumerate(groups):
    x = [point.x for point in group.members]
    y = [point.y for point in group.members]
    plt.scatter(x, y, color=colors[i], label=f"Group {i+1}")
# plt.legend()
plt.show()

## 2.5 Optimization of obtained points

## 3. extract the centroids (average points) of the point groups and use them as generating points for Voronoi tessellation;


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

members = np.concatenate([group.members for group in groups])
points = [[member.x, member.y] for member in members]

centroids = [group.centroid for group in groups]
centroids = [[centroid.x, centroid.y] for centroid in centroids]

# Calculate Voronoi diagram
vor = Voronoi(centroids)

# Plot the Voronoi diagram
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='grey', line_width=0.5, point_size=2)

# Color the Voronoi cells according to their corresponding cluster centroid
for i, centroid in enumerate(centroids):
    cluster_points = points[np.argwhere(np.all(centroids[i] == centroid, axis=1)).flatten()]
    region_indices = vor.point_region[np.argwhere(np.all(points == centroid, axis=1)).flatten()]
    regions = [vor.regions[index] for index in region_indices if index != -1]
    voronoi_plot_2d(vor, show_vertices=False, line_colors='k', line_width=2, point_size=0)
    for region in regions:
        polygon = [vor.vertices[i] for i in region]
        plt.fill(*zip(*polygon), alpha=0.2, label=f'Cluster {i+1}')
    plt.scatter(cluster_points[:,0], cluster_points[:,1], color=f'C{i}', label=f'Cluster {i+1} Points')
    
# Add title and legend
plt.title('Voronoi Cells for Clustered Data')
plt.legend(loc='upper left')

# Show the plot
plt.show()


In [ ]:
n_c = 4
n_r = 20
list_of_num = [[]*n_c] * n_r

len(list_of_num[0])